In [63]:
import requests 
from bs4 import BeautifulSoup 
import csv 
from datetime import date,timedelta
from datetime import datetime
import numpy as np
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

# Retrieve Data from Intentionalist: Takeout and Delivery Directory
# URL: https://intentionalist.com/covid-19-takeout-delivery-directory/
## Author: Jeffrey Lai

# Read data from yesterday's list. If file is not found, send Email from Gmail to remind the developer. 

In [108]:
try:
    pre_day = pd.read_csv('C:\\Users\\jeffr\\takeout_{}.csv'.format(datetime.strftime(date.today()- timedelta(1),'%Y%m%d')),encoding='utf-8-sig')
except Exception:
    print('not found')
    # create message object instance
    msg = MIMEMultipart()
    message = "File Not Found"
    # setup the parameters of the message
    with open('pd.txt') as f:
        pw = f.readline()
    password = pw
    msg['From'] = "sender"
    msg['To'] = "recipient"
    msg['Subject'] = "Subscription"
    msg.attach(MIMEText(message, 'plain'))

    #create server
    server = smtplib.SMTP('smtp.gmail.com: 587')

    server.starttls()

    server.login(msg['From'], password)

    server.sendmail(msg['From'], msg['To'], msg.as_string())

    server.quit()

    print ("successfully sent email to %s:" % (msg['To']))

not found
successfully sent email to jeff257@uw.edu:


In [109]:
URL = "https://intentionalist.com/covid-19-takeout-delivery-directory/"
r = requests.get(URL)

In [110]:
soup = BeautifulSoup(r.content, 'html5lib') 

In [111]:
table = soup.findAll('div', attrs = {'class':'b-name'})

# Build Restaurant Name List

In [112]:
rest = []
for i in table:
    #rest.appendtable[3].text
    rest.append(i.text)
rest = list(map(lambda st : str.replace(st,"+","\+"),rest))

# Build Restaurant Status List

In [113]:
status = []
st = soup.findAll('div', attrs = {'class':'b-status'})
for i in st:
    status.append(i.text)

# Build Restaurant Service List

In [114]:
serv = []
service = (soup.findAll('div', attrs = {'class':'b-services'}))
for i in service:
    serv.append(i.text[8:100])

In [115]:
import re
t = soup.find('div', text = re.compile('Good Egg'),attrs = {'class':'b-name'}).find_next_siblings("div")

# Retrieve Restaurant Location

In [116]:
arr = []
rest2 = []
for store in rest:
    rest2.append(store.replace("\\",""))
    t = soup.find('div', text = re.compile(store),attrs = {'class':'b-name'})#.find_next_siblings("div")
    if t is None :
        arr.append("NaN")
    elif t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).text == "Neighborhood" and len(t.find_next_siblings("div")) == 2:
        arr.append(t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).next_sibling) # some restaurants only have neighborhood
    elif t.find_next_siblings("div")[1].find(attrs = {'class':'b-label'}).text == "Neighborhood" and len(t.find_next_siblings("div")) > 2:
        arr.append(t.find_next_siblings("div")[2].find(attrs = {'class':'b-label'}).next_sibling)
    else:
        t2 = t.find_next_siblings("div")
        arr.append(t2[1].find(attrs = {'class':'b-label'}).next_sibling)
    


# Combine All Restaurant Information into Data Frame

In [117]:
import pandas as pd
df = pd.DataFrame(arr,rest2)


In [118]:
df['status'] = status

In [119]:
df['service'] = serv

In [120]:
df = df.reset_index().rename(columns={'index':'name', 0:"address"})

In [121]:
df[df['name'].str.contains("Hong Kong")]

,name,address,status,service
5,A + Hong Kong Kitchen,"419 6th Ave S, Seattle, WA 98104",,"Takeout, Delivery (via 3rd Party)"
311,Hong Kong Bistro,"507 Maynard Ave S #511C, Seattle, WA 98104",,Takeout


# Extract Data for restaurants in Washington State only

In [122]:
df["address"] = df['address'].str.strip()
df["state"] = df['address'].str.extract('(WA|Seattle|([9][8][0-9]+)$)')[0]
#df[df['state'].isnull()]

In [123]:
df["state"] = np.where(df["name"].isin([ "Shiro's Sushi"]),"WA",df["state"])

# Write 2 Data files To local Hard Drive. List of new restaurants and a complete list of restaurants.

In [124]:
try:
    pre_day = pd.read_csv('C:\\Users\\jeffr\\takeout_{}.csv'.format(datetime.strftime(date.today()- timedelta(1),'%Y%m%d')),encoding='utf-8-sig')
    result = pd.merge(pre_day.rename(columns={'name':'name1'}), df, left_on='name1', right_on='name', how='right')
    new_rest = result[result['name1'].isnull()][['name','address_y','status_y','service_y','state_y']]
    new_rest.to_csv('C:\\Users\\jeffr\\new_restaurant_{}.csv'.format(datetime.strftime(date.today(),'%Y%m%d')),encoding='utf-8-sig')
    df.to_csv('C:\\Users\\jeffr\\takeout_{}.csv'.format(datetime.strftime(date.today(),'%Y%m%d')),encoding='utf-8-sig') # complete list
except Exception:
    df.to_csv('C:\\Users\\jeffr\\takeout_{}.csv'.format(datetime.strftime(date.today(),'%Y%m%d')),encoding='utf-8-sig')